<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
step4 : rfm
</font>
</h1>

# RFM
RFM is a method used for analyzing customer value and segmenting customers which is commonly used in database marketing and direct marketing. It has received particular attention in the retail and professional services industries.[1]

RFM stands for the three dimensions:

* Recency : How recently did the customer purchase?
* Frequency : How often do they purchase?
* Monetary Value : How much do they spend?

These criteria indicate important indicators in customer behavior. For example, the more recent the purchase, the more likely the customer will buy again. The more times a customer buys, the more satisfied the customer is, and the amount of revenue a customer generates for the company can differentiate him from other buyers.

In this step, we should calculate these three criteria for each customer (<code>CustomerId</code>).

after calculate score of each factor, we will label them by their quartile like below:
<ul>
        <li><b>Group1</b>: value <= Q1</li>
        <li><b>Group2</b>: Q1 &lt; value <= Q2</li>
        <li><b>Group3</b>: Q2 &lt; value <= Q3</li>
        <li><b>Group4</b>: Q3 &lt; value</li>
</ul>

Using these three criteria, the value of each customer (RFM criterion) is calculated by putting Recency-Frequency-Monetary value groups together. For example, look at the examples below.

   
| CustomerID | Recency | Frequency | Monetary_value| R_quartile      | F_quartile      | M_quartile      | RFM    |
| ---------- | ------- | --------- | -------- | ------ | ------ | ------ | ------ |
| 12346      | 165     | 11        | 372.86   | 4      | 4      | 2      | 442    |
| 12347      | 3       | 2         | 1323.32  | 1      | 2      | 3      | 123    |
| 12348      | 74      | 1         | 222.16   | 3      | 1      | 1      | 311    |


after that we can group customers with their **RFM score**: 


    
| Segment | RFM |
|:-------------:|:------|
| Best | 144 |
| AlmostLost | 344 |  
| LostBigSpenders | 444  | 
| LostCheap | 441 |
| Loyal | X4X | 
| BigSpenders | XX4 | 

In [1]:
# import libraries: TODO
import pandas as pd

In [2]:
# read the preprocessed dataset: TODO
df = pd.read_csv("preprocessed_sales.csv")
df

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom
...,...,...,...,...,...,...,...,...
400911,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530,United Kingdom
400912,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530,United Kingdom
400913,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530,United Kingdom
400914,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530,United Kingdom


calculate **Frequency, Recency, Monetary** value for each customer.

In [6]:
frequency_list=df.drop_duplicates("InvoiceNumber").groupby("CustomerId")["InvoiceNumber"].count()
frequency_list

CustomerId
12346    11
12347     2
12348     1
12349     3
12351     1
         ..
18283     6
18284     1
18285     1
18286     2
18287     4
Name: InvoiceNumber, Length: 4312, dtype: int64

In [16]:
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"])
today=df["InvoiceDate"].max()
recency_list = today - df.groupby("CustomerId")["InvoiceDate"].max()
recency_list = recency_list.dt.days+1
recency_list

CustomerId
12346    165
12347      3
12348     74
12349     43
12351     11
        ... 
18283     18
18284     67
18285    296
18286    112
18287     18
Name: InvoiceDate, Length: 4312, dtype: int64

In [8]:
df["TotalPrice"] = df["UnitPrice"]*df["Quantity"]
monetary_list = df.groupby("CustomerId")["TotalPrice"].sum()
monetary_list

CustomerId
12346     372.86
12347    1323.32
12348     222.16
12349    2671.14
12351     300.93
          ...   
18283     619.37
18284     461.68
18285     427.00
18286    1296.43
18287    2345.71
Name: TotalPrice, Length: 4312, dtype: float64

In [20]:
customer_df = pd.concat([recency_list , frequency_list , monetary_list] , axis=1)
customer_df.columns=["recency" , "frequency" , "monetary"]
customer_df

,recency,frequency,monetary
CustomerId,,,
12346,165,11,372.86
12347,3,2,1323.32
12348,74,1,222.16
12349,43,3,2671.14
12351,11,1,300.93
...,...,...,...
18283,18,6,619.37
18284,67,1,461.68
18285,296,1,427.00


now we can find group number of each factor

In [17]:
def group_def(arr, q1 , q2 , q3 ):
    if (arr<=q1):
        return 1
    elif (arr>q1) & (arr<=q2):
        return 2
    elif (arr>q2) & (arr<=q3):
        return 3
    elif (arr>q3):
        return 4
            
    

In [23]:

customer_df["R_quartile"]= customer_df["recency"].apply(group_def ,args=(customer_df["recency"].quantile(0.25) \
                                                                         ,customer_df["recency"].quantile(0.5) \
                                                                         ,customer_df["recency"].quantile(0.75)) )

customer_df["F_quartile"]= customer_df["frequency"].apply(group_def , args=(customer_df["frequency"].quantile(0.25) \
                                                                            ,customer_df["frequency"].quantile(0.5) \
                                                                            ,customer_df["frequency"].quantile(0.75)) )

customer_df["M_quartile"]= customer_df["monetary"].apply(group_def , args=(customer_df["monetary"].quantile(0.25) \
                                                                           ,customer_df["monetary"].quantile(0.5) \
                                                                           ,customer_df["monetary"].quantile(0.75)) )

customer_df

,recency,frequency,monetary,R_quartile,F_quartile,M_quartile
CustomerId,,,,,,
12346,165,11,372.86,4,4,2
12347,3,2,1323.32,1,2,3
12348,74,1,222.16,3,1,1
12349,43,3,2671.14,2,3,4
12351,11,1,300.93,1,1,1
...,...,...,...,...,...,...
18283,18,6,619.37,1,4,2
18284,67,1,461.68,3,1,2
18285,296,1,427.00,4,1,2


now we can find **RFM score** : 

In [24]:
def rank_def(arr):
    arr=int(arr)
    if(arr==144):
        return "Best"
    elif (arr==344):
        return "AlmostLost"
    elif (arr==444):
        return "LostBigSpenders"
    elif (arr == 441):
        return "LostCheap"
    elif ( (arr//10)%10 ==4):
        return "Loyal"
    elif (arr%10 ==4):
        return "BigSpenders"
    else:
        return "Normal"

In [26]:
customer_df["RFM"] = customer_df["R_quartile"]*100+customer_df["F_quartile"]*10+customer_df["M_quartile"]
customer_df["RFM"].apply(rank_def)
customer_df

,recency,frequency,monetary,R_quartile,F_quartile,M_quartile,RFM
CustomerId,,,,,,,
12346,165,11,372.86,4,4,2,442
12347,3,2,1323.32,1,2,3,123
12348,74,1,222.16,3,1,1,311
12349,43,3,2671.14,2,3,4,234
12351,11,1,300.93,1,1,1,111
...,...,...,...,...,...,...,...
18283,18,6,619.37,1,4,2,142
18284,67,1,461.68,3,1,2,312
18285,296,1,427.00,4,1,2,412


In [27]:
df_customer_segments = pd.DataFrame(customer_df["RFM"].apply(rank_def))
df_customer_segments.reset_index(inplace=True)
df_customer_segments.columns=["CustomerId","Segment"]
df_customer_segments

,CustomerId,Segment
0,12346,Loyal
1,12347,Normal
2,12348,Normal
3,12349,BigSpenders
4,12351,Normal
...,...,...
4307,18283,Loyal
4308,18284,Normal
4309,18285,Normal
4310,18286,Normal
